In [1]:
import time

import cv2
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision

In [2]:
from ObjectDetection.voc import make_filepath_list, GetBBoxAndLabel, DataTransform, PreprocessVOC2012, multiobject_collate_fn
from ObjectDetection.ssd import SSD, MultiBoxLoss

In [3]:
DEVICE = "cuda"

# Data

In [4]:
# VOC2012の正解ラベルのリスト
voc_classes = [
    'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
    'bus', 'car', 'cat', 'chair', 'cow',
    'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train',
    'tvmonitor']

# データセットのGBR
color_mean = (104, 117, 123)

# 画像を入力サイズを300x300
input_size = 300

batch_size = 48

In [5]:
train_img_list, train_anno_list, val_img_list, val_anno_list = make_filepath_list(
    "./data/voc2012/VOC2012/"
)

train_dataset = PreprocessVOC2012(
    train_img_list,  # イメージのパスリスト
    train_anno_list, # アノテーションのパスリスト
    phase="train",   # 訓練モード
    transform=DataTransform(input_size, color_mean), # 前処理オブジェクト
    get_bbox_label=GetBBoxAndLabel(voc_classes)) # BBoxとラベル取得

val_dataset = PreprocessVOC2012(
    val_img_list,  # イメージのパスリスト
    val_anno_list, # アノテーションのパスリスト
    phase="val",   # 訓練モード
    transform=DataTransform(input_size, color_mean), # 前処理オブジェクト
    get_bbox_label=GetBBoxAndLabel(voc_classes),) # BBoxとラベル取得


# 訓練用のミニバッチを生成するデータローダー
train_dataloader = DataLoader(
    train_dataset,         # 前処理した訓練データ
    batch_size=batch_size, # バッチサイズ
    shuffle=True, # ミニバッチ抽出の際にシャッフルする
    collate_fn=multiobject_collate_fn) # ミニバッチ生成関数

# 検証用のミニバッチを生成するデータローダー
val_dataloader = DataLoader(
    val_dataset,           # 前処理した検証データ
    batch_size=batch_size, # バッチサイズ
    shuffle=False, # ミニバッチ抽出の際にシャッフルしない
    collate_fn=multiobject_collate_fn) # ミニバッチ生成関数

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

In [6]:
for batch in val_dataloader:
    break

In [7]:
# plt.imshow(cv2.cvtColor(cv2.imread(val_img_list[4]), cv2.COLOR_BGR2RGB))

In [8]:
len(batch)

2

In [9]:
# tuple[0]resizeされた画像
batch[0].shape

torch.Size([48, 3, 300, 300])

In [10]:
batch[0][1].shape

torch.Size([3, 300, 300])

In [11]:
# tuple[1]はground truth
# [cx, cy, w, h, クラス]
# 例えば画像2枚目は2つの物体がある
batch[1][1]

tensor([[ 0.0900,  0.0300,  0.9980,  0.9970, 18.0000],
        [ 0.1220,  0.5676,  0.1640,  0.7267, 14.0000]])

# モデル

In [12]:
ssd_cfg = {
    'classes_num': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'dbox_num': [4, 6, 6, 6, 4, 4],  # DBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSDモデルのvgg以外のネットワークの重みはHeの初期値で初期化
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  # バイアス項がある場合
            nn.init.constant_(m.bias, 0.0)

net = SSD(
    phase='train', # 訓練モード
    cfg=ssd_cfg
)   # 設定値のdictオブジェクト

net.vgg.load_state_dict(torch.load("./data/vgg16_reducedfc.pth"))

# Heの初期値を適用
net.extras.apply(weights_init) # extrasネットワーク
net.loc.apply(weights_init)    # locネットワーク
net.conf.apply(weights_init)   #confネットワーク

ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [13]:
_ = net.to(DEVICE)

# Train

In [14]:
# 損失関数のオブジェクトを生成
criterion = MultiBoxLoss(
    jaccard_thresh=0.5, # 背景のDBoxに分類するときのIoUの閾値
    neg_pos=3, # 背景のDBoxの数はPositive DBoxの何倍にするか
    device=DEVICE) # ネットワークのTensorに割り当てるデバイス

In [ ]:
num_epochs = 30
train_steps = len(dataloaders_dict["train"].dataset)//batch_size

# 勾配降下アルゴリズムを使用するオプティマイザーを生成
optimizer = optim.SGD(
    net.parameters(),  # SSDモデルのパラメーター
    lr=1e-3,           # 学習率
    momentum=0.9,      # 慣性項に割り当てる係数
    weight_decay=5e-4) # 重み更新時のL2正則化の係数

for epoch in range(num_epochs+1):
    print('---------------------------------------')
    print('Epoch {}/{}'.format(epoch, num_epochs))
    print('---------------------------------------')
    
    # train
    net.train()
    epoch_train_loss = 0.
    epoch_val_loss = 0.
    for batch_idx, (images, targets) in enumerate(dataloaders_dict["train"]):
        images = images.to(DEVICE)
        targets = [t.to(DEVICE) for t in targets]
        
        optimizer.zero_grad()
        
        outputs = net(images)
        
        loss_l, loos_c = criterion(outputs, targets)
        loss = loss_l + loos_c
        
        loss.backward()
        
        nn.utils.clip_grad_value_(net.parameters(), clip_value=2.0)
        
        optimizer.step()
        
        if batch_idx == 0 or batch_idx % 10 == 0:
            print(f"【epoch】:{epoch}, 【step】:{batch_idx}/{train_steps}, loss: {loss.item():.4f}")
        
        epoch_train_loss += loss.item()
    
    # validation
    net.eval()
    if (epoch == 0) or epoch % 10 == 0:
        for images, targets in dataloaders_dict["val"]:
            images = images.to(DEVICE)
            targets = [t.to(DEVICE) for t in targets]
            
            outputs = net(images)
            
            loss_l, loos_c = criterion(outputs, targets)
            loss = loss_l + loos_c
            
            epoch_val_loss += loss.item()
            
    
    print(f"【epoch end】:{epoch},【train loss】: {epoch_train_loss:.4f}, 【val loss】: {epoch_val_loss:.4}")
torch.save(net.state_dict(), "./ssd_weight.ph")

---------------------------------------
Epoch 0/30
---------------------------------------
【epoch】:0, 【step】:0/119, loss: 26.9424
【epoch】:0, 【step】:10/119, loss: 15.1216
【epoch】:0, 【step】:20/119, loss: 12.1537
【epoch】:0, 【step】:30/119, loss: 10.3944
【epoch】:0, 【step】:40/119, loss: 8.5421
【epoch】:0, 【step】:50/119, loss: 9.2846
【epoch】:0, 【step】:60/119, loss: 8.7512
【epoch】:0, 【step】:70/119, loss: 8.3279
【epoch】:0, 【step】:80/119, loss: 8.2127
【epoch】:0, 【step】:90/119, loss: 8.2097
【epoch】:0, 【step】:100/119, loss: 8.9272
【epoch】:0, 【step】:110/119, loss: 8.3313
【epoch end】:0,【train loss】: 1222.4488, 【val loss】: 961.2
---------------------------------------
Epoch 1/30
---------------------------------------
【epoch】:1, 【step】:0/119, loss: 7.9264
【epoch】:1, 【step】:10/119, loss: 7.9211
【epoch】:1, 【step】:20/119, loss: 7.6182
【epoch】:1, 【step】:30/119, loss: 7.9670
【epoch】:1, 【step】:40/119, loss: 7.9442
【epoch】:1, 【step】:50/119, loss: 8.1955
【epoch】:1, 【step】:60/119, loss: 8.5226
【epoch】:1, 【step